### Reading all DICOM files
#### As the file paths mentioned in CSV files and acutal file paths are different.
### ------------------------------------------------------------------------- o --------------------------------------------------------------------------------

In [226]:
import pydicom as dicom
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import PIL # optional
import pandas as pd
import csv
import re
%matplotlib inline

In [227]:
os.getcwd()

'E:\\notebooks\\stage_2'

In [282]:
root_dir = 'E:/MTP_work/stage_2/dataset/CBIS-DDSM/'

# mass_test_dir = '/home/Drive/avineil/Mass_Test_ROI/CBIS-DDSM'
# mass_train_dir = '/home/Drive/avineil/Mass_Train_ROI/CBIS-DDSM'
# calc_test_dir = '/home/Drive/avineil/Calc_Test_ROI/CBIS-DDSM'
# calc_train_dir = '/home/Drive/avineil/Calc_Train_ROI/CBIS-DDSM'


# curr_dir = calc_train_dir # change this directory accordingly
root_dir

'E:/MTP_work/stage_2/dataset/CBIS-DDSM/'

In [283]:
dcm_files = []
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".dcm"):
             dcm_files.append((os.path.join(root, file)))

In [284]:
len(dcm_files)

47

In [285]:
dcm_files[:5]

['E:/MTP_work/stage_2/dataset/CBIS-DDSM/Mass-Training_P_01888_RIGHT_MLO_1\\07-21-2016-DDSM-98952\\1-ROI mask images-23508\\000000.dcm',
 'E:/MTP_work/stage_2/dataset/CBIS-DDSM/Mass-Training_P_01888_RIGHT_MLO_1\\07-21-2016-DDSM-98952\\1-ROI mask images-23508\\000001.dcm',
 'E:/MTP_work/stage_2/dataset/CBIS-DDSM/Mass-Training_P_01889_RIGHT_CC\\07-20-2016-DDSM-51519\\1-full mammogram images-76495\\000000.dcm',
 'E:/MTP_work/stage_2/dataset/CBIS-DDSM/Mass-Training_P_01889_RIGHT_CC_1\\07-21-2016-DDSM-46689\\1-ROI mask images-49263\\000000.dcm',
 'E:/MTP_work/stage_2/dataset/CBIS-DDSM/Mass-Training_P_01889_RIGHT_CC_1\\07-21-2016-DDSM-46689\\1-ROI mask images-49263\\000001.dcm']

### ------------------------------------------------------------------------- o --------------------------------------------------------------------------------
### Conerting DICOM files to PNG files.

In [317]:
def fileTypeChecker(dcm_file):
    """ Returns dicom file type of the given file as one of the following [full_mammogram, ROI_patch, Mask]"""
    
    img = dicom.dcmread(dcm_file)
    img_path = dcm_file
    try:
        if img.SeriesDescription == 'cropped images':
            return 'ROI_patch', img.PatientID
        elif img.SeriesDescription == 'ROI mask images':
            return 'Mask', img.PatientID
        elif img.SeriesDescription == 'full mammogram images':
            return 'full_mammogram', img.PatientID + "_1" # "_1" to make label prediction easy when looking up in csv file.
        
    except AttributeError :
        if img_path.find('mammogram') != -1:
            return 'full_mammogram', img.PatientID + "_1" # "_1" to make label prediction easy when looking up in csv file.
        elif img_path.find('ROI') != -1:
            return 'Mask', img.PatientID

In [296]:
for f in dcm_files:
    g = dicom.dcmread(f)
    print(g.PatientID)

Mass-Training_P_01888_RIGHT_MLO_1
Mass-Training_P_01888_RIGHT_MLO_1
Mass-Training_P_01889_RIGHT_CC
Mass-Training_P_01889_RIGHT_CC_1
Mass-Training_P_01889_RIGHT_CC_1
Mass-Training_P_01889_RIGHT_MLO
Mass-Training_P_01889_RIGHT_MLO_1
Mass-Training_P_01889_RIGHT_MLO_1
Mass-Training_P_01890_LEFT_MLO
Mass-Training_P_01890_LEFT_MLO_1
Mass-Training_P_01890_LEFT_MLO_1
Mass-Training_P_01908_LEFT_CC
Mass-Training_P_01908_LEFT_CC_1
Mass-Training_P_01908_LEFT_CC_1
Mass-Training_P_01946_RIGHT_CC
Mass-Training_P_01946_RIGHT_CC_1
Mass-Training_P_01946_RIGHT_CC_1
Mass-Training_P_01946_RIGHT_MLO
Mass-Training_P_01946_RIGHT_MLO_1
Mass-Training_P_01946_RIGHT_MLO_1
Mass-Training_P_01981_RIGHT_CC
Mass-Training_P_01981_RIGHT_CC_1
Mass-Training_P_01981_RIGHT_CC_1
Mass-Training_P_01981_RIGHT_MLO
Mass-Training_P_01981_RIGHT_MLO_1
Mass-Training_P_01981_RIGHT_MLO_1
Mass-Training_P_01983_LEFT_MLO
Mass-Training_P_01983_LEFT_MLO_1
Mass-Training_P_01983_LEFT_MLO_1
Mass-Training_P_02033_RIGHT_CC
Mass-Training_P_02033_

In [297]:
dicom.dcmread(dcm_files[0])

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.9590.100.1.2.196340980611987253508274758753288908702
(0008, 0020) Study Date                          DA: '20160720'
(0008, 0023) Content Date                        DA: '20160426'
(0008, 0030) Study Time                          TM: '215115'
(0008, 0033) Content Time                        TM: '150020.346000'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'MG'
(0008, 0064) Conversion Type                     CS: 'WSD'
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 103e) Series Description                  LO: 'cropped images'
(0010, 0010) Patient's Name                      PN: 'Mass-Training_P_01888_RIGHT_MLO_1'
(0010, 0020) Patient ID                          LO: 'Mass-Training_P_01888_RIGHT_ML

In [299]:
root_dir

'E:/MTP_work/stage_2/dataset/CBIS-DDSM/'

In [300]:
def writeFileToDisk(dir_name, file_name, label, image_file):
    img = dicom.dcmread(image_file).pixel_array
    img = img - img.min()
    img = img /(img.max() - img.min())
    img *= 255
    img = np.uint8(img)
    cv2.imwrite(dir_name+ file_name + "_" + label + ".png" ,img) # write png image
    

In [318]:
ROI_patch_dir = 'E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/ROI_patches/'
full_mammogram_dir = 'E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/full_mammograms/'
Mask_dir = 'E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/masks/'

ROI_count, full_mammogram_count, mask_count = (0, 0, 0)

for f in dcm_files:
    if fileTypeChecker(f)[0] == 'ROI_patch':
        print(ROI_patch_dir + fileTypeChecker(f)[1]+'.png')
        writeFileToDisk(ROI_patch_dir, fileTypeChecker(f)[1], getLabel(f), f)
        ROI_count += 1
        
    elif fileTypeChecker(f)[0] == 'Mask':
        print(Mask_dir + fileTypeChecker(f)[1]+'.png')
        writeFileToDisk(Mask_dir, fileTypeChecker(f)[1], getLabel(f), f)
        mask_count += 1
        
    elif fileTypeChecker(f)[0] == 'full_mammogram':
        print(full_mammogram_dir + fileTypeChecker(f)[1]+'.png')
        writeFileToDisk(full_mammogram_dir, fileTypeChecker(f)[1], getLabel(f), f)
        full_mammogram_count += 1
        
    else:
        print("Unknown file type:",f)
        #cv2.imwrite(ROI_patch_dir+ fileTypeChecker(f)[1] + ".png" ,img) # write png image
else:
    print('ROIs = {}, Masks = {}, full_mammograms = {}'.format(ROI_count, mask_count, full_mammogram_count))
    print('total_images = {}'.format(len(dcm_files)))
    if ROI_count + mask_count + full_mammogram_count == len(dcm_files):
        print('All files have been Successfully written to disk!!!')
    else:
        print('Some files have not been properly processed.!!!')

E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/ROI_patches/Mass-Training_P_01888_RIGHT_MLO_1.png
E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/masks/Mass-Training_P_01888_RIGHT_MLO_1.png
E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/full_mammograms/Mass-Training_P_01889_RIGHT_CC_1.png
E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/ROI_patches/Mass-Training_P_01889_RIGHT_CC_1.png
E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/masks/Mass-Training_P_01889_RIGHT_CC_1.png
E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/full_mammograms/Mass-Training_P_01889_RIGHT_MLO_1.png
E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/ROI_patches/Mass-Training_P_01889_RIGHT_MLO_1.png
E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/masks/Mass-Training_P_01889_RIGHT_MLO_1.png
E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/full_mammograms/Mass-Training_P_01890_LEFT_MLO_1.png
E:/MTP_work/stage_2/dataset/CBIS-DDSM/processed_images/masks/Mass

In [302]:
dex = pd.read_csv('E:/MTP_work/stage_2/dataset/mass_case_description_train_set.csv')

In [316]:
dex.loc[dex['patient_id'] == 'P_01889']

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
1303,P_01889,1,RIGHT,CC,1,mass,LOBULATED,CIRCUMSCRIBED,3,BENIGN,5,Mass-Training_P_01889_RIGHT_CC/1.3.6.1.4.1.959...,Mass-Training_P_01889_RIGHT_CC_1/1.3.6.1.4.1.9...,Mass-Training_P_01889_RIGHT_CC_1/1.3.6.1.4.1.9...
1304,P_01889,1,RIGHT,MLO,1,mass,LOBULATED,CIRCUMSCRIBED,3,BENIGN,5,Mass-Training_P_01889_RIGHT_MLO/1.3.6.1.4.1.95...,Mass-Training_P_01889_RIGHT_MLO_1/1.3.6.1.4.1....,Mass-Training_P_01889_RIGHT_MLO_1/1.3.6.1.4.1....


In [271]:
name = dex.loc[2]
name

patient_id                                                           P_00038
breast density                                                             2
left or right breast                                                   RIGHT
image view                                                                CC
abnormality id                                                             1
abnormality type                                               calcification
calc type                                                           VASCULAR
calc distribution                                                        NaN
assessment                                                                 2
pathology                                            BENIGN_WITHOUT_CALLBACK
subtlety                                                                   5
image file path            Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...
cropped image file path    Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....

In [272]:
patient_name = '{patient_id}_{left or right breast}_{image view}_{abnormality id}_{pathology}'.format(**name)
patient_name

'P_00038_RIGHT_CC_1_BENIGN_WITHOUT_CALLBACK'

In [309]:
n = fileTypeChecker(dcm_files[0])

In [310]:
n[1]

'Mass-Training_P_01888_RIGHT_MLO_1'

In [312]:
PID , LorR, view, abnormality_id = n[1].split("_")[-4:]
(PID , LorR, view, abnormality_id)

('01888', 'RIGHT', 'MLO', '1')

In [313]:
label = dex.loc[(dex['patient_id'] == 'P_'+PID) & (dex['left or right breast'] == LorR) & (dex['image view'] == view) & (dex['abnormality id'] == int(abnormality_id)) ]['pathology']
label

1302    MALIGNANT
Name: pathology, dtype: object

In [275]:
label.values.item()

'BENIGN_WITHOUT_CALLBACK'

In [292]:
def getLabel(img):
    name = fileTypeChecker(img)
    PID , LorR, view, abnormality_id = name[1].split("_")[-4:]
    label = dex.loc[(dex['patient_id'] == 'P_'+PID) & (dex['left or right breast'] == LorR) & (dex['image view'] == view) & (dex['abnormality id'] == int(abnormality_id)) ]['pathology']
    if label.values.item() == 'BENIGN_WITHOUT_CALLBACK':
        return 'BENIGN' # considering BENIGN_WITHOUT_CALLBACK same as BENIGN
    return label.values.item()
    